In [1]:
# -*- coding: utf-8 -*-
import time
import random
import pickle
import queue
import numpy as np

In [2]:
#   Función que devuelve una matriz de adyacencia de un grafo ponderado (con pesos)
#   con n_nodes, una proporción sparse_factor de ramas (?¿) y max_weight como peso máximo
def rand_matr_pos_graph(n_nodes, sparse_factor, max_weight=50., decimals=0):    ## TODO: Usar el decimals

    matBinaria = np.random.binomial(1, sparse_factor, (n_nodes, n_nodes))           # Creamos la matriz que contiene las conexiones
    matPesos = np.random.binomial(max_weight, sparse_factor, (n_nodes, n_nodes))    # Creamos la matriz que contiene los pesos
    matAdyacencia = np.empty((n_nodes, n_nodes))                                    # Creamos la matriz de adyacencia
    matAdyacencia.fill(np.inf)                                                      # La rellenamos de infinito

    matFinal = matBinaria * matPesos            # Multiplicamos la primera matriz por la segunda para tener los pesos SOLO en las conexiones generadas

    for i in range(matAdyacencia.shape[0]):     # Recorremos la matriz de adyacencia
        for j in range(matAdyacencia.shape[1]):
            if i == j:                                  # Para poner a 0 la diagonal
                matAdyacencia[i][j] = 0
            elif matFinal[i][j] != 0:                   # Y para rellenar las conexiones
                matAdyacencia[i][j] = matFinal[i][j]

    return matAdyacencia

In [3]:
#   Funcion que devuelve el numero de ramas en el grafo dada una matriz de adyacencia
def cuenta_ramas(m_g):

    num_ramas = 0

    for filas in m_g:                           # Bucle que recorre la matriz de adyacencia
        for peso in filas:
            if peso != 0 and peso != np.inf:            # Sumando 1 cuando encuentra una conexión
                num_ramas += 1

    return num_ramas

In [4]:
#   Función que genera las matrices de n_grafos aleatorios con n_nodes y un cierto
#   sparse_factor, devolviendo la media de sparse_factor reales de las matrices generadas
def check_sparse_factor(n_grafos, n_nodes, sparse_factor):

    sp_aux = 0          # Inicializamos a 0 esta variable auxiliar

    for i in range(n_grafos):       # Hacemos un bucle tantas veces como grafos tengamos que generar
        mat = rand_matr_pos_graph(n_nodes, sparse_factor)   # Generamos la matriz de adyacencia
        sp = cuenta_ramas(mat)                              # Contamos las ramas de la matriz generada
        sp_aux += sp                                        # Sumamos las ramas al contador

    avg_sparse_factor = sp_aux/n_grafos     # Por ultimo, calculamos el factor de dispersion medio

    return avg_sparse_factor

In [5]:
#   Función que devuelve el diccionario de listas de adyacencia
#   del grafo decinido por la matriz de adyacencia m_g
#   SE SUPONE QUE TIENE QUE GENERAR UN DICCIONARIO DE UNA MATRIZ (?¿)
def m_g_2_d_g(m_g):

    d_g = {}    # Diccionario a devolver
    index = 0   # Indice para el diccionario G (indice de nodo)

    for nodo in m_g:        # Bucle que recorre la matriz

        d_g[index] = {}     # Diccionario G[i]
        j = 0       # Indice para el diccionario G[i] (no. de nodo)

        for peso in nodo:                       # Recorremos los valores de cada fila (nodo)
            if peso != 0 and peso != np.inf:        # Si tiene peso, lo añadimos al diccionario
                d_g[index][j] = peso                    # j porque pienso que los nodos van de 0-inf, no de 1-inf
            j+=1

        index += 1          # Añadimos 1 al indice del diccionario G

    return d_g

In [6]:
#   Función que devuelve una matriz de adyacencia
#   dado un diccionario de listas de adyacencia d_g
def d_g_2_m_g(d_g):

    m_g = np.empty((len(d_g), len(d_g)))    # Como son matrices cuadradas, la longitud del diccionario nos da la cantidad de nodos
    m_g.fill(np.inf)                        # La rellenamos de np.inf

    for i in range(len(d_g)):               # Bucle para inicializar la diagonal a 0
        m_g[i][i] = 0

    for k1, v1 in d_g.items():              # Recorremos el diccionario. k1 = indice ; v1 = dic. de ese nodo
        for k2, v2 in v1.items():               # Recorremos el diccionario del nodo
            m_g[k1][k2] = v2                        # Colocamos el peso en la posición que le corresponde

    return m_g

In [7]:
m=rand_matr_pos_graph(5,0.5)
print(m)
n=cuenta_ramas(m)
print(n)
s=check_sparse_factor(10,5,0.5)
print(s)
d=m_g_2_d_g(m)
print(d)
mm=d_g_2_m_g(d)
print(mm)

[[ 0. inf inf inf inf]
 [inf  0. 28. 23. inf]
 [27. 29.  0. inf inf]
 [25. inf inf  0. 25.]
 [inf inf 27. inf  0.]]
7
9.2
{0: {}, 1: {2: 28.0, 3: 23.0}, 2: {0: 27.0, 1: 29.0}, 3: {0: 25.0, 4: 25.0}, 4: {2: 27.0}}
[[ 0. inf inf inf inf]
 [inf  0. 28. 23. inf]
 [27. 29.  0. inf inf]
 [25. inf inf  0. 25.]
 [inf inf 27. inf  0.]]


In [8]:
#   Función que guarda un objeto Python obj de manera comprimida en un fichero de nombre f_name
def save_object(obj, f_name='obj.pklz', save_path='.'):

    objFile = open(save_path + f_name, 'wb')    # Abrimos el fichero en modo de escritura binaria para que funcione pickle
    pickle.dump(obj, objFile)                   # Guardamos el objeto ¿ya serializado? en el fichero
    objFile.close()

In [9]:
#   Función que devuelve un objeto Python guardado en un fichero de nombre f_name
def read_object(f_name, save_path='.'):

    objFile = open(save_path + f_name, 'rb')    # Abrimos el fichero en modo de lectura binaria para que funcione pickle
    object = pickle.load(fp)                    # Cargamos el objeto ¿serializado? guardado en el fichero
    objFile.close()                             # Cerramos el fichero

    return object

In [10]:
#   Función que escribe en un fichero de nombre f_name un grafo ponderado en formato
#   TGF a partir de un diccionario de listas de adyacencia
def d_g_2_TGF(d_g, f_name):

    TGFFile = open(f_name, 'w')                 # Abrimos el fichero en modo escritura

    for indice in d_g:                          # Escribimos primero los indices en el fichero
        TGFFile.write(indice + '\n')

    TGFFile.write('#\n')                        # Escribimos el separador

    for nodoOrg, DiccDestinos in d_g:           # Recorremos de nuevo el diccionario para escribir
        for nodoDst, pesoRec in DiccDestinos:       # el resto del fichero
            TGFFile.write(nodoOrg + ' ' + nodoDst + ' ' + pesoRec + '\n')   # Escribimos los diferentes datos

    TGFFile.close() 

In [11]:
#   Función que devuelve un diccionario de listas de adyacencia a partir de un
#   grafo ponderado TGF guardado en el archivo f_name   ## TODO: Se guarda como str en el dicc
def TGF_2_d_g(f_name):

    d_g = {}                                    # Inicializamos el diccionario de listas de adyacencia

    TGFFile = open(f_name, 'r')                 # Abrimos nuestro fichero donde tenemos el grafo en TGF

    for linea in TGFFile:                       # Leemos cada linea del fichero para obtener los indices de los diferentes nodos
        indice = linea.split('\n')[0]               # Dividimos la linea y cogemos el indice del nodo
        if indice == '#':                           # Cuando leemos el '#' salimos del bucle ya que hemos leido todos los indices
            break
        else:                                       # Si no leemos '#', guardamos el indice y creamos un diccionario para el mismo
            d_g[indice] = {}

    for linea in TGFFile:                       # Ahora guardamos en los distintos diccionarios los nodos destino y los costes hacia ellos
        nodoOrg, nodoDst, pesoRec = linea.split('\n')[0].split(' ')    # Fragmentamos la linea leida en los diferentes datos
        d_g[nodoOrg][nodoDst] = pesoRec                                # Creamos una clave y valor nuevos para el diccionario de nodoOrg

    TGFFile.close()                             # Cerramos el fichero

    return d_g

In [12]:
m_g = rand_matr_pos_graph(5,0.25)
print(m_g)
d_g = m_g_2_d_g(m_g)
print(d_g)

[[ 0. inf inf 13. inf]
 [inf  0. inf inf 18.]
 [12. inf  0. inf  7.]
 [inf  8. 17.  0.  8.]
 [13. inf inf  9.  0.]]
{0: {3: 13.0}, 1: {4: 18.0}, 2: {0: 12.0, 4: 7.0}, 3: {1: 8.0, 2: 17.0, 4: 8.0}, 4: {0: 13.0, 3: 9.0}}


In [13]:
#   Función que aplica Dijkstra a un nodo u dentro de un grafo d_g en formato de
#   diccionario. La funcion devuelve dos diciconarios, d_prev donde las claves
#   son los nodos previos a las valores, y d_dist, tiene como clave un nodo
#   accesible partiendo desde u y como valores el coste minimo total hasta ese nodo
def dijkstra_d(d_g, u):

    lstOpenNodes = [] #s    # Lista de nodos abiertos
    d_dist = {} #d          # Diccionario de costes minimos
    d_prev = {} #p          # Diccionario de nodos previos

    for i in range(len(d_g)):       # Inicializamos la lista entera a False
        lstOpenNodes.append(False)  ## TODO: ¿Se puede hacer mejor?

    Q = queue.PriorityQueue()       # Inicializamos la cola de prioridad

    d_dist[u] = 0           # K : nodoDst ; V : peso
    Q.put((d_dist[u],u))    # [0]: Peso; [1]: NodoDst   # Debe ser asi para que funcione la prioridad

    # Mientras la cola de prioridad no este vacia
    while not Q.empty():

        dist_tuple = Q.get()        # Cogemos la tupla (peso, distancia)
        dist = dist_tuple[0]
        nodoActual = dist_tuple[1]

        #   Si el nodo no esta cerrado
        if not lstOpenNodes[nodoActual]:

            lstOpenNodes[nodoActual] = True             # Cerramos el nodo
            diccionario_adyacencias = d_g[nodoActual]   # Sacamos las adyacencias del nodo a analizar

            # Sacamos las conexiones
            for dicc_NodeDst, dicc_Dist in diccionario_adyacencias.items():

                #   Si no existe, lo creamos
                if not dicc_NodeDst in d_dist:
                    d_dist[dicc_NodeDst] = dicc_Dist + dist     # Nueva Entrada. {v : (dist(prev(v),v) + Dst_Acumulada)}
                    d_prev[dicc_NodeDst] = nodoActual           # Nueva Entrada. {v : prev(v)}
                    Q.put((d_dist[dicc_NodeDst],dicc_NodeDst))  # Metemos elemento en Q. (dist(u,v), v)

                #   Si existe, comprobamos si tiene menor peso que el anterior guardado
                elif d_dist[dicc_NodeDst] > (d_dist[nodoActual] + d_g[nodoActual][dicc_NodeDst]):
                    d_dist[dicc_NodeDst] = d_dist[nodoActual] + d_g[nodoActual][dicc_NodeDst]   # Nueva Entrada. {v : (dist(prev(v),v) + Dst_Acumulada)}
                    d_prev[dicc_NodeDst] = nodoActual           # Nueva Entrada. {v : prev(v)}
                    Q.put((d_dist[dicc_NodeDst],dicc_NodeDst))  # Metemos elemento en Q. (dist(u,v), v)

    return d_dist, d_prev

In [14]:
def dijkstra_m(m_g, u):
    pass

In [15]:
def min_paths(d_prev):
    pass

In [16]:
#   Función que genera n_graphs número de grafos con un numero de nodos entre n_nodes_ini
#   y n_nodes_fin con pasos de step nodos. Despues, para cada grafo generado, aplicamos
#   el algoritmo de Dijkstra para cada nodo de cada grafo, midiendo el tiempo en el
#   se resuelve el mismo. Los grafos están en modo Matriz Numpy
def time_dijkstra_m(n_graphs, n_nodes_ini, n_nodes_fin, step, sparse_factor=.25):
    pass

In [17]:
#   Función que genera n_graphs número de grafos con un numero de nodos entre n_nodes_ini
#   y n_nodes_fin con pasos de step nodos. Despues, para cada grafo generado, aplicamos
#   el algoritmo de Dijkstra para cada nodo de cada grafo, midiendo el tiempo en el
#   se resuelve el mismo. Los grafos están en modo Diccionario
def time_dijkstra_d(n_graphs, n_nodes_ini, n_nodes_fin, step, sparse_factor=.25):

    lista_diccGrafos = []   # Creamos una lista vacia para los grafos en formato diccionario

    # Creamos n_graphs grafos con los distintos valores entre n_nodes_ini y n_nodes_fin
    # menores estrictamente que n_nodes_fin debido a como funciona range()
    for n_nodes in range(n_nodes_ini, n_nodes_fin, step):
        for j in range(n_graphs):
            newMatrixGraph = rand_matr_pos_graph(n_nodes, sparse_factor)
            newDiccGraph = m_g_2_d_g(newMatrixGraph)
            lista_diccGrafos.append(newDiccGraph)
    # Creamos los grafos con n_nodes_fin numero de nodos que falta
    for j in range(n_graphs):
        newMatrixGraph = rand_matr_pos_graph(n_nodes_fin, sparse_factor)
        newDiccGraph = m_g_2_d_g(newMatrixGraph)
        lista_diccGrafos.append(newDiccGraph)

    listaStep_diccTiempos = []      # Creamos una lista auxiliar vacia para los tiempos de resolución de cada matriz
    lista_diccAvgTiempos = []       # Creamos una lista vacia para los tiempos de resolución de cada matriz
    actualNodes_step = n_nodes_ini  # Inicializamos el step actual al minimo de nodos

    for grafoDicc in lista_diccGrafos:

        time_start = time.time()            # Tiempo en segundos al empezar a resolver un grafo
        for nodo_ini in grafoDicc:          # Resolvemos el grafo para TODOS los nodos
            dijkstra_d(grafoDicc, nodo_ini)
        time_end = time.time()              # Tiempo en segundos al finalizar de resolver un grafo

        # Calculamos el tiempo medio de resolucion del algoritmo para todos los grafos de un step
        # Por tanto, solo entremos sí el grafo actual es de distinto tamaño al step actual o si
        # estamos en el ultimo grafo, en ese caso calculariamos el tiempo medio de resolución para
        # el ultimo step
        if (len(grafoDicc) != actualNodes_step) or (grafoDicc == lista_diccGrafos[-1]):

            if grafoDicc == lista_diccGrafos[-1]:   # Añadimos el ultimo nodo
                listaStep_diccTiempos.append(time_end - time_start)

            avg_tiempo = 0  # Inicializamos el tiempo medio
            for tiempo in listaStep_diccTiempos:    # Hacemos el sumatorio de todos los tiempos de un step
                avg_tiempo += tiempo
            avg_tiempo /= n_graphs                  # Calculamos la media

            lista_diccAvgTiempos.append(avg_tiempo)     # Añadimos a la lista el nuevo tiempo medio
            actualNodes_step += step                    # Aumentamos el step
            listaStep_diccTiempos = []                  # Limpiamos la lista de tiempos del step

        listaStep_diccTiempos.append(time_end - time_start) # Añadimos el nuevo tiempo a la lista de un step

    return lista_diccAvgTiempos

In [18]:
lista_tiemposDicc = time_dijkstra_d(5, 10, 20, 5, sparse_factor=.25)
print(lista_tiemposDicc)

[0.0027876853942871093, 0.007494783401489258, 0.014191961288452149]
